In [72]:
import pandas as pd 
import numpy as np

Steam is the leading PC platform for games and has millions of reviews around its games. 

The dataset contains over 6.4 million publicly available reviews in English from Steam Reviews portion of Steam store run by Valve. Each review is described by review text, the id of game it belongs to, the name of the game, review sentiment (positive or negative) and whether the review was recommended by another user or not

In [10]:
# Reading the CSV file
file_path = 'dataset/dataset-kaggle.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

In [11]:
# Displaying the DataFrame
df.head()  # Displays the first 5 rows of the DataFrame

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [29]:
df.shape

(6417106, 5)

In [15]:
df['app_id'].nunique()  # Count of unique app IDs

9972

Check for missing values

In [12]:
# Checking for missing values in the entire DataFrame
missing_values = df.isnull().sum()

# Displaying the count of missing values for each column
print("Missing values per column:")
print(missing_values)

Missing values per column:
app_id               0
app_name        183234
review_text       7305
review_score         0
review_votes         0
dtype: int64


Since app_name is required so that we can link it to another dataset that has the tags describing the game, let'

In [16]:
# Filtering rows where the 'app_name' column has missing values
missing_app_name_entries = df[df['app_name'].isnull()]

# Displaying the rows with missing 'app_name'
print("Entries with missing 'app_name':")
missing_app_name_entries.head(10)

Entries with missing 'app_name':


,app_id,app_name,review_text,review_score,review_votes
21883,100980,NaN,"If you're an amatuer, who knows nothing about ...",1,1
21884,100980,NaN,3D coat is must have app for 3D modeling guys....,1,1
21885,100980,NaN,Relatively powerful tool. The voxel painting i...,1,1
21886,100980,NaN,"Amazing piece of software, I mainly use it for...",1,1
21887,100980,NaN,"High quality software, as fun to use as it is ...",1,1
21888,100980,NaN,Well worth the $79.99 purchase. The program wo...,1,1
21889,100980,NaN,Brilliant modelling program. It is on par with...,1,0
21890,100980,NaN,Fantastic program at an accessible price for a...,1,1
21891,100980,NaN,If you want one package to cover the whole mod...,1,1
21892,100980,NaN,"The more I use it, the more I love it. Tons of...",1,1


In [17]:
missing_app_name_entries['app_id'].nunique()  # Count of unique app IDs with missing 'app_name'

604

# Wilson Scoring (Entire Dataset)

Create a new dataframe where each entry has the app_id, game_id, number of votes, number of downvotes and lastly number of upvotes

In [56]:
# Reading the CSV file
file_path = 'dataset/dataset-kaggle.csv'
df_games_votes = pd.read_csv(file_path)

# Grouping by the game and calculating the counts
game_stats = df_games_votes.groupby('app_id').agg(
    total_votes=('review_score', 'count'),
    upvotes=('review_score', lambda x: (x == 1).sum()),
    downvotes=('review_score', lambda x: (x == -1).sum())
).reset_index()

In [57]:
# Displaying the resulting DataFrame
game_stats.head()# Displays the first 5 rows of the grouped DataFrame

,app_id,total_votes,upvotes,downvotes
0,10,12353,11965,388
1,20,1542,1294,248
2,30,1030,937,93
3,40,457,389,68
4,50,1743,1678,65


Add app_name to corresponding app_id

In [58]:
# Adding the app_name for each app_id without repetition
app_names = df[['app_id', 'app_name']].drop_duplicates(subset='app_id')  # Ensure unique app_id-app_name pairs

In [59]:
app_names

,app_id,app_name
0,10,Counter-Strike
12353,1002,Rag Doll Kung Fu
12395,100400,Silo 2
12428,10090,Call of Duty: World at War
21883,100980,NaN
...,...,...
6414010,99810,Bulletstorm
6416169,99830,Crysis 2
6416238,9990,Ghostbusters: Sanctum of Slime
6416307,99900,Spiral Knights


In [60]:
game_stats = game_stats.merge(app_names, on='app_id', how='left')

In [61]:
game_stats.head(10)  # Displaying the first 5 rows of the merged DataFrame

,app_id,total_votes,upvotes,downvotes,app_name
0,10,12353,11965,388,Counter-Strike
1,20,1542,1294,248,Team Fortress Classic
2,30,1030,937,93,Day of Defeat
3,40,457,389,68,Deathmatch Classic
4,50,1743,1678,65,Half-Life: Opposing Force
5,60,1237,1040,197,Ricochet
6,70,7916,7649,267,Half-Life
7,80,2122,1900,222,Counter-Strike: Condition Zero
8,130,1344,1216,128,Half-Life: Blue Shift
9,220,6483,6480,3,Half-Life 2


In [62]:
game_stats.shape

(9972, 5)

In [64]:
# moving app_name to column next to app_id
game_stats = game_stats[['app_id', 'app_name', 'total_votes', 'upvotes', 'downvotes']]

In [66]:
game_stats.head(10)

,app_id,app_name,total_votes,upvotes,downvotes
0,10,Counter-Strike,12353,11965,388
1,20,Team Fortress Classic,1542,1294,248
2,30,Day of Defeat,1030,937,93
3,40,Deathmatch Classic,457,389,68
4,50,Half-Life: Opposing Force,1743,1678,65
5,60,Ricochet,1237,1040,197
6,70,Half-Life,7916,7649,267
7,80,Counter-Strike: Condition Zero,2122,1900,222
8,130,Half-Life: Blue Shift,1344,1216,128
9,220,Half-Life 2,6483,6480,3


In [67]:
# Saving the DataFrame to a JSON file
game_stats.to_json('game_stats.json', orient='records', lines=True)

Loading the saved JSON file

In [69]:
# Loading the JSON file back into a DataFrame
df_loaded = pd.read_json('game_stats.json', orient='records', lines=True)

# Displaying the loaded DataFrame
df_loaded.head()

,app_id,app_name,total_votes,upvotes,downvotes
0,10,Counter-Strike,12353,11965,388
1,20,Team Fortress Classic,1542,1294,248
2,30,Day of Defeat,1030,937,93
3,40,Deathmatch Classic,457,389,68
4,50,Half-Life: Opposing Force,1743,1678,65


The Wilson Score is defined by - 

$$
\text{Wilson Score}=\frac{p + \frac{z^2}{2n} - z\sqrt{\frac{p(1 - p) + \frac{z^2}{4n}}{n}}}{1 + \frac{z^2}{n}}
$$

In [70]:
def wilson_score(upvotes, total_votes, z=1.96):
    """
    Calculate the Wilson score interval for a given number of positive votes and total votes.

    The Wilson score is used to estimate a confidence interval for a proportion and is used for ranking items 
    based on user feedback.

    Parameters:
    - upvotes (int): The number of positive votes, the user recommended this game
    - total_votes (int): The total number of votes (upvote + downvote).
    - z (float): The z-score corresponding to the desired confidence level. 
      Default is 1.96, which corresponds to a 95% confidence level.

    Returns:
    - float: The lower bound of the Wilson score interval, representing the adjusted proportion
      of positive votes with the specified confidence level.
    """
    
    wilson_score = 0
    if total_votes != 0:
        p = upvotes / total_votes
        denominator = 1 + z ** 2 / total_votes
        numerator_sum_term = p + z ** 2 / (2 * total_votes)
        numerator_square_root_term = np.sqrt(
            (p * (1 - p) + (z ** 2 / (4 * total_votes))) / total_votes
        )
        wilson_score = (
            (numerator_sum_term - z * numerator_square_root_term) / denominator
        )   
    return wilson_score

Calculate Wilson Scores for each Entry

In [73]:
# apply the function to each row (axis=1) of the DataFrame
game_stats['wilson_score'] = game_stats.apply(
    lambda row: wilson_score(row['upvotes'], row['total_votes']),
    axis=1
)

In [74]:
game_stats.head()  # Displaying the first 5 rows of the DataFrame with Wilson score

,app_id,app_name,total_votes,upvotes,downvotes,wilson_score
0,10,Counter-Strike,12353,11965,388,0.965366
1,20,Team Fortress Classic,1542,1294,248,0.819994
2,30,Day of Defeat,1030,937,93,0.890650
3,40,Deathmatch Classic,457,389,68,0.815651
4,50,Half-Life: Opposing Force,1743,1678,65,0.952747


In [75]:
game_stats_sorted = game_stats.sort_values(by='wilson_score', ascending=False).reset_index(drop=True)

In [76]:
game_stats_sorted.shape

(9972, 6)

In [78]:
game_stats_sorted.head(5)  # Displaying the first 10 rows of the sorted DataFrame

,app_id,app_name,total_votes,upvotes,downvotes,wilson_score
0,8930,Sid Meier's Civilization V,8693,8690,3,0.998986
1,6060,"Star Wars: Battlefront 2 (Classic, 2005)",2976,2976,0,0.998711
2,220,Half-Life 2,6483,6480,3,0.998640
3,213670,South Park™: The Stick of Truth™,3479,3475,4,0.997047
4,200260,Batman: Arkham City GOTY,868,868,0,0.995594


Save the final dataframe as JSON file

In [79]:
# Saving the DataFrame to a JSON file
game_stats_sorted.to_json('game_stats_wilson.json', orient='records', lines=True)

In [81]:
# Loading the JSON file back into a DataFrame
df_wilson = pd.read_json('game_stats_wilson.json', orient='records', lines=True)

In [82]:
df_wilson.head(10)

,app_id,app_name,total_votes,upvotes,downvotes,wilson_score
0,8930,Sid Meier's Civilization V,8693,8690,3,0.998986
1,6060,"Star Wars: Battlefront 2 (Classic, 2005)",2976,2976,0,0.998711
2,220,Half-Life 2,6483,6480,3,0.998640
3,213670,South Park™: The Stick of Truth™,3479,3475,4,0.997047
4,200260,Batman: Arkham City GOTY,868,868,0,0.995594
5,339800,HuniePop,2831,2825,6,0.995383
6,235460,METAL GEAR RISING: REVENGEANCE,1196,1195,1,0.995279
7,431240,Golf With Your Friends,520,520,0,0.992666
8,257510,The Talos Principle,1024,1020,4,0.989999
9,411960,TY the Tasmanian Tiger,957,953,4,0.989302
